Esta nota utiliza métodos vistos en [1.5.Integracion_numerica](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.5.Integracion_numerica.ipynb)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_numerical -p 8888:8888 -p 8786:8786 -p 8787:8787 -d palmoreck/jupyterlab_numerical:1.1.0
```

Detener el contenedor de docker:

```
docker stop jupyterlab_local
```


Documentación de la imagen de docker `palmoreck/jupyterlab_numerical:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/numerical).

---

Instalamos las herramientas que nos ayudarán al perfilamiento:

In [1]:
%pip install -q --user line_profiler

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q --user memory_profiler

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q --user psutil

Note: you may need to restart the kernel to use updated packages.


La siguiente celda reiniciará el kernel de **IPython** para cargar los paquetes instalados en la celda anterior. Dar **Ok** en el mensaje que salga y continuar con el contenido del notebook.

In [4]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import math
import numpy as np

from scipy.integrate import quad
import matplotlib.pyplot as plt

# Perfilamiento en Python

En esta nota revisamos algunas herramientas de Python para perfilamiento de código: uso de cpu y memoria.

Medición de tiempos con:

* Módulo [time](https://docs.python.org/3/library/time.html#time.time) de Python.

* [%time](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time) de comandos de magic <- esta herramienta es sólo para medir tiempos de un statement y sólo la coloco para referencia pero no se usará en la nota.

* [/usr/bin/time](https://en.wikipedia.org/wiki/Time_(Unix)) de `Unix`.

* [%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) de comandos de magic.

Perfilamiento:

* De CPU con: [line_profiler](https://pypi.org/project/line-profiler/), [CProfile](https://docs.python.org/2/library/profile.html) que es `built-in` en la *standard-library* de Python.

* De memoria con: [memory_profiler](https://pypi.org/project/memory-profiler/).

## Medición de tiempos

El primer acercamiento que usamos en la nota para perfilar nuestro código es identificar qué es lento, otras mediciones son la cantidad de RAM, el I/O en disco o network. 

### 1) Uso de `time`

In [2]:
import time

### Regla compuesta del rectángulo

**Ejemplo de implementación de regla compuesta de rectángulo: usando math**

Utilizar la regla compuesta del rectángulo para aproximar la integral $\int_0^1e^{-x^2}dx$ con $10^6$ subintervalos.

In [3]:
f=lambda x: math.exp(-x**2) #using math library

In [4]:
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

In [5]:
n=10**6

In [6]:
start_time = time.time()
aprox=Rcf(f,0,1,n)
end_time = time.time()

In [7]:
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.4476315975189209 segundos


**Obs:** recuérdese que hay que evaluar que se esté resolviendo correctamente el problema. En este caso el error relativo nos ayuda

In [8]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [9]:
obj, err = quad(f, 0, 1)
err_relativo(aprox,obj)

6.71939731300312e-14

**Comentarios:**

* Tómese en cuenta que al medir tiempos de ejecución, siempre hay variación en la medición. Tal variación es normal.

* Considérese que la máquina en la que se están corriendo las pruebas puede estar realizando otras tareas mientras se ejecuta el código, por ejemplo acceso a la red, al disco o a la RAM. Por ello, son factores que pueden causar variación en el tiempo de ejecución del programa.

* Si se van a realizar reportes de tiempos, es importante indicar las características de la máquina en la que se están haciendo las pruebas, p.ej: Dell E6420 con un procesador Intel Core I7-2720QM (2.20 GHz, 6 MB cache, Quad Core) y 8 GB de RAM en un Ubuntu $13.10$.

### 2) Uso de `/usr/bin/time` de Unix

Para la línea de comando `/usr/bin/time` primero escribimos el siguiente archivo en la ruta donde se encuentra este notebook con la línea de comando magic `%file`

In [11]:
%%file Rcf.py
import math
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf (float) 
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res
if __name__=="__main__": #añadimos este bloque para ejecución de la función Rcf
        n=10**6
        f=lambda x: math.exp(-x**2)
        print("aproximación: {:0.6e}".format(Rcf(f,0,1,n)))

Writing Rcf.py


Ver [liga](https://stackoverflow.com/questions/419163/what-does-if-name-main-do) y [liga2](https://es.stackoverflow.com/questions/32165/qu%C3%A9-es-if-name-main) para saber qué es lo que hace la línea `if __name__ == "__main__":`

Lo siguiente es necesario si no tienen instalado el comando `/usr/bin/time`:

In [12]:
%%bash
sudo apt-get install time

Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  time
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 26.2 kB of archives.
After this operation, 79.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 time amd64 1.7-25.1build1 [26.2 kB]
Fetched 26.2 kB in 2s (17.4 kB/s)
Selecting previously unselected package time.
(Reading database ... 18521 files and directories currently installed.)
Preparing to unpack .../time_1.7-25.1build1_amd64.deb ...
Unpacking time (1.7-25.1build1) ...
Setting up time (1.7-25.1build1) ...


debconf: delaying package configuration, since apt-utils is not installed


In [13]:
%%bash
/usr/bin/time -p python3 Rcf.py #la p es de portabilidad, 
                                #ver: http://manpages.ubuntu.com/manpages/xenial/man1/time.1.html
                                #para mayor información

aproximación: 7.468241e-01


real 0.40
user 0.33
sys 0.05


**Comentarios:**

* `real` que mide el wall clock o elapsed time.
* `user` que mide la cantidad de tiempo de tu ejecución que la CPU gastó para funciones que no están relacionadas con el kernel del sistema.
* `sys` que mide la cantidad de tiempo de tu ejecución que la CPU gastó en funciones a nivel de kernel del sistema.

**Obs:** Una función relacionada con el kernel del sistema es el alojamiento de memoria al crear una variable.

* La ventaja de `/usr/bin/time` es que no es específico de Python.
* Este comando incluye el tiempo que le toma al sistema iniciar el ejecutable de python (que puede ser significativo si se inician muchos procesos vs un sólo proceso). En el caso de tener short-running scripts donde el tiempo de inicio es significativo del tiempo total entonces `/usr/bin/time` puede ser una medida útil.


**Nota:** Si se suma `user` con `sys` se tiene una idea de cuánto tiempo se gastó en la CPU y la diferencia entre este resultado y `real` da una idea de cuánto tiempo se gastó para I/O o también puede dar una idea de la cantidad de tiempo que se ocupó el sistema en correr otras tareas.



* Se puede utilizar la flag `verbose` para obtener más información:

In [14]:
%%bash
/usr/bin/time --verbose python3 Rcf.py

aproximación: 7.468241e-01


	Command being timed: "python3 Rcf.py"
	User time (seconds): 0.42
	System time (seconds): 0.04
	Percent of CPU this job got: 95%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.48
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 41160
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 9083
	Voluntary context switches: 25
	Involuntary context switches: 15
	Swaps: 0
	File system inputs: 16
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


y una explicación (breve) del output se puede encontrar [aquí](http://manpages.ubuntu.com/manpages/xenial/man1/time.1.html). Para el caso de `Major (requiring I/O)` nos interesa que sea $0$ pues indica que el sistema operativo tiene que cargar páginas de datos del disco pues tales datos ya no residen en RAM (por alguna razón).

### 3) Uso de `timeit`

El módulo de `timeit` es otra forma de medir el tiempo de ejecución en la CPU.

**Nota:** el módulo de `timeit` desabilita temporalmente el garbage collector* de Python (esto es, no habrá desalojamiento en memoria de objetos de Python que no se utilicen). Si el garbage collector es invocado en tus operaciones para un ejemplo del mundo real, esto puede ser una razón de posibles diferencias que obtengas en las mediciones de tiempo. 

*sugiero buscar qué es el garbage collector en blogs, por ejemplo: [liga](https://rushter.com/blog/python-garbage-collector/) o [liga2](https://stackify.com/python-garbage-collection/) o [liga3](https://stackoverflow.com/questions/4484167/python-garbage-collector-documentation).

In [15]:
%timeit?

Docstring:
Time execution of a Python statement or expression

Usage, in line mode:
  %timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] statement
or in cell mode:
  %%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] setup_code
  code
  code...

Time execution of a Python statement or expression using the timeit
module.  This function can be used both as a line and cell magic:

- In line mode you can time a single-line statement (though multiple
  ones can be chained with using semicolons).

- In cell mode, the statement in the first line is used as setup code
  (executed but not timed) and the body of the cell is timed.  The cell
  body has access to any variables created in the setup code.

Options:
-n<N>: execute the given statement <N> times in a loop. If <N> is not
provided, <N> is determined so as to get sufficient accuracy.

-r<R>: number of repeats <R>, each consisting of <N> loops, and take the
best result.
Default: 7

-t: use time.time to measure the time, which is the default on Unix.
This f

In [16]:
%timeit -n 5 -r 10 Rcf(f,0,1,n)

335 ms ± 6.44 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


para este caso se está ejecutando la función `Rcf` en un loop de tamaño $5$, se están promediando los tiempos de las $5$ ejecuciones y calculando su desviación estándar y al repetir esto $10$ veces se está reportando el mejor resultado. $ms$ es milisecond, $\mu s$ es microsecond y $ns$ es nanosecond.

**Comentarios:**

* `timeit` se recomienda usar para secciones de código pequeñas. Para secciones más grandes típicamente modificar el valor de $n$ (ejecutar el código n veces en un loop) resulta en mediciones distintas.

* Ejecuta `timeit` varias ocasiones para asegurarse que se obtienen tiempos similares. Si observas una gran variación en las mediciones de tiempo entre distintas repeticiones de `timeit`, realiza más repeticiones hasta tener un resultado estable.

### 4) Uso de cProfile

`cProfile` es una herramienta **built-in** en la standard library para perfilamiento. Se utiliza con la implementación `CPython` de `Python` (ver [liga](https://stackoverflow.com/questions/17130975/python-vs-cpython) para explicación de implementaciones de Python) para medir el tiempo de ejecución de cada función en el programa.

Se ejecuta desde la línea de comandos. La flag `-s` indica que se ordene el resultado por el tiempo acumulado dentro de cada función.

El output siguiente de `cProfile` muestra:

* El tiempo total de ejecución, el cual incluye el tiempo del bloque de código que estamos midiendo y el overhead al usar `cProfile`. Por esta razón se tiene un mayor tiempo de ejecución que con las mediciones de tiempo anteriores.

* La columna `ncalls` que como el nombre indica, muestra el número de veces que se llamó a cada función. En este caso las funciones `lambda` y `math.exp` son las que se llaman un mayor número de veces: $n=10^6$ veces. La columna`tottime` muestra el tiempo que tardaron estas funciones en ejecutarse (sin llamar a otras funciones).

* La columna `percall` es el cociente entre `tottime` y `ncalls`.

* La columna `cumtime` contiene el tiempo gastado en la función y en las demás que llama. Por ejemplo la función `Rcf` llama a `listcomp` por lo que es natural que `Rcf` esté más arriba en el output ordenado de `cProfile`. Esto también ocurre con `lambda` y `math.exp` pues la primera llama a la segunda.

* La columna de `percall` es un cociente entre la columna `cumtime` y el llamado a primitivas.

* La última columna indica información de la función y la línea en la que se encuentra dentro del código. Por ejemplo la línea $1$ de módulo es el llamado a la función `__main__`. La línea $2$ es el llamado a la función `Rcf`. Por lo que es prácticamente negligible el llamado a `__main__`.

In [17]:
%%bash
python3 -m cProfile -s cumulative Rcf.py

aproximación: 7.468241e-01
         2000068 function calls in 0.570 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.570    0.570 {built-in method builtins.exec}
        1    0.014    0.014    0.570    0.570 Rcf.py:1(<module>)
        1    0.134    0.134    0.556    0.556 Rcf.py:2(Rcf)
  1000000    0.217    0.000    0.296    0.000 Rcf.py:23(<lambda>)
        1    0.126    0.126    0.126    0.126 Rcf.py:16(<listcomp>)
  1000000    0.079    0.000    0.079    0.000 {built-in method math.exp}
        1    0.000    0.000    0.001    0.001 <frozen importlib._bootstrap>:966(_find_and_load)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:936(_find_and_load_unlocked)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:651(_load_unlocked)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:564(module_from_spec)
        1    0

**Nota:** Recordar que el output de `CProfile` con la flag `-s cumulative` está ordenando por el gasto en tiempo de las funciones que son llamadas en el bloque de código analizado. No está ordenando por parent functions. Para tener un output en el que se tenga qué funciones llaman a qué otras se puede utilizar lo siguiente:

In [18]:
%%bash
python3 -m cProfile -o profile.stats Rcf.py

aproximación: 7.468241e-01


In [19]:
import pstats

In [20]:
p = pstats.Stats("profile.stats")
p.sort_stats("cumulative")

In [21]:
p.print_stats()

Sun Jan 19 16:25:38 2020    profile.stats

         2000068 function calls in 0.544 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.544    0.544 {built-in method builtins.exec}
        1    0.014    0.014    0.544    0.544 Rcf.py:1(<module>)
        1    0.133    0.133    0.529    0.529 Rcf.py:2(Rcf)
  1000000    0.215    0.000    0.294    0.000 Rcf.py:23(<lambda>)
        1    0.102    0.102    0.102    0.102 Rcf.py:16(<listcomp>)
  1000000    0.078    0.000    0.078    0.000 {built-in method math.exp}
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:966(_find_and_load)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:936(_find_and_load_unlocked)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:651(_load_unlocked)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:564(module_from_spec

In [22]:
p.print_callers()

   Ordered by: cumulative time

Function                                                      was called by...
                                                                  ncalls  tottime  cumtime
{built-in method builtins.exec}                               <- 
Rcf.py:1(<module>)                                            <-       1    0.014    0.544  {built-in method builtins.exec}
Rcf.py:2(Rcf)                                                 <-       1    0.133    0.529  Rcf.py:1(<module>)
Rcf.py:23(<lambda>)                                           <- 1000000    0.215    0.294  Rcf.py:2(Rcf)
Rcf.py:16(<listcomp>)                                         <-       1    0.102    0.102  Rcf.py:2(Rcf)
{built-in method math.exp}                                    <- 1000000    0.078    0.078  Rcf.py:23(<lambda>)
<frozen importlib._bootstrap>:966(_find_and_load)             <-       1    0.000    0.000  Rcf.py:1(<module>)
<frozen importlib._bootstrap>:936(_find_and_load_unlocked)    

y podemos también tener la información de a qué funciones llamó cada función

In [23]:
p.print_callees()

   Ordered by: cumulative time

Function                                                      called...
                                                                  ncalls  tottime  cumtime
{built-in method builtins.exec}                               ->       1    0.014    0.544  Rcf.py:1(<module>)
Rcf.py:1(<module>)                                            ->       1    0.000    0.000  <frozen importlib._bootstrap>:966(_find_and_load)
                                                                       1    0.133    0.529  Rcf.py:2(Rcf)
                                                                       1    0.000    0.000  {built-in method builtins.print}
                                                                       1    0.000    0.000  {method 'format' of 'str' objects}
Rcf.py:2(Rcf)                                                 ->       1    0.102    0.102  Rcf.py:16(<listcomp>)
                                                                 1000000    0.21

---

### 5) Uso de line_profiler en python: %lprun

`line_profiler` trabaja perfilando el código de forma individual funciones línea por línea. La idea sería perfilar primero con `CProfile` al programa para identificar aquellas funciones que gastan un mayor tiempo de ejecución y posteriormente perfilarlas con `line_profiler`.

**Comentario:** una buena práctica es guardar las diferentes versiones de tu código cuando vas modificándolo para tener un registro de tus cambios.

Puede ejecutarse desde la línea de comandos o cargarse en IPython con el comando magic `load_ext`: 

In [24]:
%load_ext line_profiler

In [25]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat

En el siguiente output:

* La columna `%Time` contiene el porcentaje de tiempo gastado. En el caso que se perfila, la línea`sum_res=sum_res+f(node)` es en la que más porcentaje del tiempo se gasta. Seguida de la línea del `for` y de la línea donde se hace uso de [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) para crear a los nodos de integración numérica.

In [26]:
%lprun -f Rcf Rcf(f,0,1,n)

Timer unit: 1e-06 s

Total time: 1.67834 s
File: <ipython-input-4-08ae95932785>
Function: Rcf at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n and h_hat=(b-a)/n
     6                                               Args:
     7                                                   f (lambda expression): lambda expression of integrand
     8                                                   a (int): left point of interval
     9                                                   b (int): right point of in

Con la evidencia generada con `line_profiler` ¿podríamos escribir una función que fuera más rápida?

Lo primero que podemos hacer es utilizar un [generator](https://wiki.python.org/moin/Generators) en lugar de una lista:

In [30]:
def Rcf2(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf2 (float) 
    """
    h_hat=(b-a)/n
    nodes=(a+(i+1/2)*h_hat for i in range(0,n))
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res

medir con `%timeit`:

In [31]:
%timeit -n 5 -r 10 Rcf2(f,0,1,n)

352 ms ± 9.65 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


revisar que está correcta esta nueva implementación:

In [32]:
err_relativo(aprox,obj)

6.71939731300312e-14

perfilarla con `line_profiler`:

In [33]:
%lprun -f Rcf2 Rcf2(f,0,1,n)

Timer unit: 1e-06 s

Total time: 1.88174 s
File: <ipython-input-30-794f7be94ca8>
Function: Rcf2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf2(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Args:
     7                                                   f (lambda expression): lambda expression of integrand
     8                                                   a (int): left point of interval
     9                                                   b (int): right point of interval
    10                                       

y observar que la línea en la que se creaba la lista ahora es despreciable el porcentaje de tiempo que se gasta en ella.

Podemos hacer una implementación que se encargue del gasto del tiempo en la línea del `for`:

In [38]:
def Rcf3(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf3 (float) 
    """
    h_hat=(b-a)/n
    nodes=(a+(i+1/2)*h_hat for i in range(0,n))
    suma_res = sum((f(node) for node in nodes))
    return h_hat*suma_res

medir con `%timeit`:

In [39]:
%timeit -n 5 -r 10 Rcf3(f,0,1,n)

371 ms ± 7.9 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


revisar que está correcta esta nueva implementación:

In [40]:
err_relativo(aprox,obj)

6.71939731300312e-14

perfilarla con `line_profiler`:

In [41]:
%lprun -f Rcf3 Rcf3(f,0,1,n)

Timer unit: 1e-06 s

Total time: 1.01353 s
File: <ipython-input-38-0b5466de5036>
Function: Rcf3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf3(f,a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Args:
     7                                                   f (lambda expression): lambda expression of integrand
     8                                                   a (int): left point of interval
     9                                                   b (int): right point of interval
    10                                       

y se tiene la mayoría del porcentaje de tiempo ahora en una sola línea.

Recuérdese que el resultado de `Cprofile` indicó que se llama a la función `lambda` y `math.exp` $n=10^6$ veces. Una implementación de la regla del rectángulo con menor número de llamadas (y por tanto menor tiempo) sería:

In [43]:
def Rcf4(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf4 (float) 
    """
    h_hat=(b-a)/n
    nodes=(a+(i+1/2)*h_hat for i in range(0,n))
    suma_res = sum(((math.exp(-node**2) for node in nodes)))
    return h_hat*suma_res

In [45]:
%lprun -f Rcf4 Rcf4(0,1,n)

Timer unit: 1e-06 s

Total time: 0.879869 s
File: <ipython-input-43-ef8e351f19bf>
Function: Rcf4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def Rcf4(a,b,n):
     2                                               """
     3                                               Compute numerical approximation using rectangle or mid-point method in 
     4                                               an interval.
     5                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     6                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     7                                               Args:
     8                                                   f (lambda expression): lambda expression of integrand
     9                                                   a (int): left p

In [46]:
%timeit -n 5 -r 10 Rcf4(0,1,n)

318 ms ± 8.76 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


Si bien esta implementación es la más rápida no es tan flexible pues está calculando la regla del rectángulo para una función definida, si quisiéramos calcular la regla para otra función se tendría que directamente modificar la función `Rcf` lo cual no es flexible. Aunque `Rcf4` es más rápida preferimos `Rcf3` por su flexibilidad.

**Ejemplo de ejecución de line_profiler desde la línea de comandos:**

In [58]:
%%file Rcf4.py
import math
@profile #esta línea es necesaria para indicar que la siguiente función 
         #desea perfilarse con line_profiler
def Rcf4(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf4 (float) 
    """
    h_hat=(b-a)/n
    nodes=(a+(i+1/2)*h_hat for i in range(0,n))
    suma_res = sum(((math.exp(-node**2) for node in nodes)))
    return h_hat*suma_res
if __name__ == "__main__":
    n=10**6
    print("aproximación: {:0.6e}".format(Rcf4(0,1,n)))

Writing Rcf4.py


In [59]:
%%bash
/home/miuser/.local/bin/kernprof -l -v Rcf4.py

aproximación: 7.468241e-01
Wrote profile results to Rcf4.py.lprof
Timer unit: 1e-06 s

Total time: 0.785749 s
File: Rcf4.py
Function: Rcf4 at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profile #esta línea es necesaria para indicar que la siguiente función 
     3                                                    #desea perfilarse con line_profiler
     4                                           def Rcf4(a,b,n):
     5                                               """
     6                                               Compute numerical approximation using rectangle or mid-point method in 
     7                                               an interval.
     8                                               Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     9                                               Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid 

Observese en el output de `CProfile` siguiente para la función `Rcf4` que las líneas con mayor gasto en el tiempo total son: 

```
    nodes=(a+(i+1/2)*h_hat for i in range(0,n))
    suma_res = sum(((math.exp(-node**2) for node in nodes)))
```

In [61]:
%%file Rcf4.py
import math
def Rcf4(a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf4 (float) 
    """
    h_hat=(b-a)/n
    nodes=(a+(i+1/2)*h_hat for i in range(0,n))
    suma_res = sum(((math.exp(-node**2) for node in nodes)))
    return h_hat*suma_res
if __name__ == "__main__":
    n=10**6
    print("aproximación: {:0.6e}".format(Rcf4(0,1,n)))

Overwriting Rcf4.py


In [62]:
%%bash
python3 -m cProfile -s cumulative Rcf4.py

aproximación: 7.468241e-01
         3000070 function calls in 0.714 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.714    0.714 {built-in method builtins.exec}
        1    0.000    0.000    0.714    0.714 Rcf4.py:1(<module>)
        1    0.000    0.000    0.713    0.713 Rcf4.py:2(Rcf4)
        1    0.083    0.083    0.713    0.713 {built-in method builtins.sum}
  1000001    0.359    0.000    0.630    0.000 Rcf4.py:18(<genexpr>)
  1000001    0.182    0.000    0.182    0.000 Rcf4.py:17(<genexpr>)
  1000000    0.089    0.000    0.089    0.000 {built-in method math.exp}
        1    0.000    0.000    0.001    0.001 <frozen importlib._bootstrap>:966(_find_and_load)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:936(_find_and_load_unlocked)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:651(_load_unlocked)
        1    0.000    0.000   

## Uso de memory_profiler: %memit y %mprun

In [31]:
%load_ext memory_profiler

In [32]:
%memit #how much RAM this process is consuming

peak memory: 97.20 MiB, increment: 0.00 MiB


In [33]:
%memit Rcf_1(f,0,1,10**5)

peak memory: 99.00 MiB, increment: 1.80 MiB


In [34]:
%memit

peak memory: 97.51 MiB, increment: 0.00 MiB


In [35]:
%memit Rcf_2(f,0,1,10**5)

peak memory: 97.51 MiB, increment: 0.00 MiB


In [36]:
%%file Rcf_1_c.py

def Rcf_1(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
    Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
    Args:
        f (lambda expression): lambda expression of integrand
        a (int): left point of interval
        b (int): right point of interval
        n (int): number of subintervals
    Returns:
        Rcf_1 (float) 
    """
    mid_point_formula=lambda a,b:a+(b-a)/2.0
    h_hat=(b-a)/n
    nodes=[mid_point_formula(a+h_hat*i, a+h_hat*(i+1)) for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res 


Writing Rcf_1_c.py


In [37]:
from Rcf_1_c import Rcf_1 as Rcf_1_c

In [38]:
%mprun -f Rcf_1_c Rcf_1_c(f,0,1,10**5)

Filename: /datos/MNO_desde_2018/ramas_repo/mno-master/temas/I.computo_cientifico/Rcf_1_c.py

Line #    Mem usage    Increment   Line Contents
     2     97.5 MiB     97.5 MiB   def Rcf_1(f,a,b,n):
     3                                 """
     4                                 Compute numerical approximation using rectangle or mid-point method in 
     5                                 an interval.
     6                                 Nodes are generated via formula: x_i = a+(b-a)/n*i for i=0,1,...,n
     7                                 Mid point is calculated via formula: x_{i-1}+(x_i-x_{i-1})/2 for i=1,...,n to avoid rounding errors
     8                                 Args:
     9                                     f (lambda expression): lambda expression of integrand
    10                                     a (int): left point of interval
    11                                     b (int): right point of interval
    12                                     n (int): number 

**Ejercicios**

1. Resuelve los ejercicios y preguntas de la nota.


**Referencias**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.